In [11]:
using KnetOnnx

In [32]:
graph = ONNXtoGraph("model_gather.onnx")
PrintGraph(graph)

model inputs: ["0"]
model outputs: ["2"]
(op1) Constant
	output1: 1
(op2) Gather
	input1: 0
	input2: 1
	output1: 2


In [33]:
model = KnetModel(graph);

In [34]:
x1 = randn(3,4)

3×4 Array{Float64,2}:
  0.642565  -0.0266282  -1.61274   0.59253 
  1.19636    1.14969     1.05971   2.10387 
 -1.52113   -2.04965    -0.885484  0.775168

In [35]:
model(x1)

2-element Array{Any,1}:
 [0.6425647273409499, 1.1963594513258913, -1.521133413460643]
 [0.6425647273409499, 1.1963594513258913, -1.521133413460643]

In [36]:
model.tensors

Dict{Any,Any} with 3 entries:
  "1" => Float32[0.0, 0.0]
  "0" => [0.642565 -0.0266282 -1.61274 0.59253; 1.19636 1.14969 1.05971 2.10387…
  "2" => Any[[0.642565, 1.19636, -1.52113], [0.642565, 1.19636, -1.52113]]